In [ ]:
import numpy as np
import pysindy as ps
from pysindy.feature_library import PolynomialLibrary, FourierLibrary
import matplotlib.pyplot as plt

from generate_data import generate_hiv_data, generate_discrete_hiv_data

In [ ]:
T = 1e-1
t = np.linspace(0, 100, 1001)
x0 = [1, 1, 1, 1, 1]
x0_val = [2, 1, 2, 2, 3]

x, x_dot = generate_hiv_data(t=t, x0=x0)
x_val, _ = generate_hiv_data(t=t, x0=x0_val)

xk = generate_discrete_hiv_data(t=t, x0=x0, T=T)
xk_val = generate_discrete_hiv_data(t=t, x0=x0_val, T=T)

In [ ]:
model = ps.SINDy(
    feature_library=PolynomialLibrary(degree=3),
    # feature_library=FourierLibrary(n_frequencies=1),
    optimizer=ps.STLSQ(threshold=0.0001),
    feature_names=[f'x{i+1}' for i in range(len(x[0]))],
    # discrete_time=True
    )
# model.fit(x=xk)
model.fit(x=x, x_dot=x_dot, t=t)
model.print(precision=3)

In [ ]:
x_sim = model.simulate(x0=x0_val, t=t)
mse = ((x_sim - xk_val)**2).mean(axis=0)
mse

In [ ]:
fig, axs = plt.subplots(5, 1, figsize=(10, 15), sharex=True)

labels = [
    "Zdrowe CD4+", "Zainfekowane CD4+", "Prekursory LTC",
    "Pomocniczo-niezależne LTC", "Pomocniczo-zależne LTC"
]
model_labels = [
    "Model zdrowych CD4+", "Model zainfekowanych CD4+", "Model prekursorów LTC",
    "Model pomocniczo-niezależnych LTC", "Model pomocniczo-zależnych LTC"
]

for i in range(5):
    axs[i].plot(t, xk_val[:, i], label=labels[i], color=f"C{2*i}")
    axs[i].plot(t, x_sim[:, i], '--', label=model_labels[i], color=f"C{2*i+1}")
    axs[i].set_ylabel("Stężenie", fontsize=16)
    axs[i].legend(fontsize=14)
    axs[i].grid()
    axs[i].tick_params(axis='both', labelsize=14)

axs[-1].set_xlabel("Czas [dni]", fontsize=16)
plt.tight_layout()
plt.show()